# Recruit Restaurant Visitor Forecasting Version II

Following:
https://www.kaggle.com/tunguz/surprise-me-2

The difficulty in my baseline model: reserved tables have many missing restaurants. How to use them? 

In [1]:
import glob, re
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor

## Load Data

In [3]:
df_ar = pd.read_csv('data/air_reserve.csv')
df_hr = pd.read_csv('data/hpg_reserve.csv')
df_astore = pd.read_csv('data/air_store_info.csv')
df_hstore = pd.read_csv('data/hpg_store_info.csv')
df_storeid = pd.read_csv('data/store_id_relation.csv')
df_av = pd.read_csv('data/air_visit_data.csv')
date_info = pd.read_csv('data/date_info.csv')
df_submission = pd.read_csv('data/sample_submission.csv')

The data description see:

https://github.com/dongzhang84/Kaggle/blob/master/Restaurant_Visitor_Forecasting/Visitor_forecasting_v1.ipynb

In [4]:
df_hr.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors
0,hpg_c63f6f42e088e50f,2016-01-01 11:00:00,2016-01-01 09:00:00,1
1,hpg_dac72789163a3f47,2016-01-01 13:00:00,2016-01-01 06:00:00,3
2,hpg_c8e24dcf51ca1eb5,2016-01-01 16:00:00,2016-01-01 14:00:00,2
3,hpg_24bb207e5fd49d4a,2016-01-01 17:00:00,2016-01-01 11:00:00,5
4,hpg_25291c542ebb3bc2,2016-01-01 17:00:00,2016-01-01 03:00:00,13


In [6]:
df_hr = pd.merge(df_hr, df_storeid, how='inner', on='hpg_store_id')
df_hr.head()

,hpg_store_id,visit_datetime,reserve_datetime,reserve_visitors,air_store_id
0,hpg_878cc70b1abc76f7,2016-01-01 19:00:00,2016-01-01 15:00:00,4,air_db80363d35f10926
1,hpg_878cc70b1abc76f7,2016-01-02 19:00:00,2016-01-02 14:00:00,2,air_db80363d35f10926
2,hpg_878cc70b1abc76f7,2016-01-03 18:00:00,2016-01-02 20:00:00,6,air_db80363d35f10926
3,hpg_878cc70b1abc76f7,2016-01-06 20:00:00,2016-01-04 22:00:00,3,air_db80363d35f10926
4,hpg_878cc70b1abc76f7,2016-01-11 18:00:00,2016-01-11 14:00:00,2,air_db80363d35f10926


In [7]:
df_ar['visit_datetime'] = pd.to_datetime(df_ar['visit_datetime'])
df_ar['visit_datetime'] = df_ar['visit_datetime'].dt.date

In [11]:
df_ar['reserve_datetime'] = pd.to_datetime(df_ar['reserve_datetime'])
df_ar['reserve_datetime'] = df_ar['reserve_datetime'].dt.date

In [13]:
df_ar['reserve_datetime_diff'] = df_ar.apply(lambda r: (r['visit_datetime'] 
                                                              - r['reserve_datetime']).days, axis=1)

In [16]:
tmp1 = df_ar.groupby(['air_store_id','visit_datetime'], as_index=False)\
       [['reserve_datetime_diff', 'reserve_visitors']].sum()\
       .rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})

In [18]:
tmp2 = df_ar.groupby(['air_store_id','visit_datetime'], as_index=False)\
       [['reserve_datetime_diff', 'reserve_visitors']].mean()\
       .rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})

In [21]:
df_ar.head()

,air_store_id,visit_datetime,reserve_datetime,reserve_visitors,reserve_datetime_diff
0,air_877f79706adbfb06,2016-01-01,2016-01-01,1,0
1,air_db4b38ebe7a7ceff,2016-01-01,2016-01-01,3,0
2,air_db4b38ebe7a7ceff,2016-01-01,2016-01-01,6,0
3,air_877f79706adbfb06,2016-01-01,2016-01-01,2,0
4,air_db80363d35f10926,2016-01-01,2016-01-01,5,0


In [23]:
df_ar = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])
df_ar.head()

,air_store_id,visit_date,rs1,rv1,rs2,rv2
0,air_00a91d42b08b08d9,2016-10-31,0,2,0.0,2.0
1,air_00a91d42b08b08d9,2016-12-05,4,9,4.0,9.0
2,air_00a91d42b08b08d9,2016-12-14,6,18,6.0,18.0
3,air_00a91d42b08b08d9,2016-12-17,6,2,6.0,2.0
4,air_00a91d42b08b08d9,2016-12-20,2,4,2.0,4.0


In [27]:
len(df_ar.air_store_id.unique())

314

In [28]:
df_hr['visit_datetime'] = pd.to_datetime(df_hr['visit_datetime'])
df_hr['visit_datetime'] = df_hr['visit_datetime'].dt.date

df_hr['reserve_datetime'] = pd.to_datetime(df_hr['reserve_datetime'])
df_hr['reserve_datetime'] = df_hr['reserve_datetime'].dt.date

In [29]:
df_hr['reserve_datetime_diff'] = df_hr.apply(lambda r: (r['visit_datetime'] 
                                             - r['reserve_datetime']).days, axis=1)

In [30]:
tmp1 = df_hr.groupby(['air_store_id','visit_datetime'], as_index=False)\
       [['reserve_datetime_diff', 'reserve_visitors']].sum()\
       .rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs1', 'reserve_visitors':'rv1'})

tmp2 = df_hr.groupby(['air_store_id','visit_datetime'], as_index=False)\
       [['reserve_datetime_diff', 'reserve_visitors']].mean()\
       .rename(columns={'visit_datetime':'visit_date', 'reserve_datetime_diff': 'rs2', 'reserve_visitors':'rv2'})

df_hr = pd.merge(tmp1, tmp2, how='inner', on=['air_store_id','visit_date'])
df_hr.head()

,air_store_id,visit_date,rs1,rv1,rs2,rv2
0,air_00a91d42b08b08d9,2016-01-14,3,2,3.0,2.0
1,air_00a91d42b08b08d9,2016-01-15,6,4,6.0,4.0
2,air_00a91d42b08b08d9,2016-01-16,3,2,3.0,2.0
3,air_00a91d42b08b08d9,2016-01-22,3,2,3.0,2.0
4,air_00a91d42b08b08d9,2016-01-29,6,5,6.0,5.0


In [31]:
len(df_hr.air_store_id.unique())

150

In [32]:
# featuring df_av

df_av['visit_date'] = pd.to_datetime(df_av['visit_date'])
df_av['dow'] = df_av['visit_date'].dt.dayofweek
df_av['year'] = df_av['visit_date'].dt.year
df_av['month'] = df_av['visit_date'].dt.month
df_av['visit_date'] = df_av['visit_date'].dt.date

In [36]:
df_av.head()

,air_store_id,visit_date,visitors,dow,year,month
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1


In [34]:
# featuring df_submission

df_submission['visit_date'] = df_submission['id'].map(lambda x: str(x).split('_')[2])
df_submission['air_store_id'] = df_submission['id'].map(lambda x: '_'.join(x.split('_')[:2]))
df_submission['visit_date'] = pd.to_datetime(df_submission['visit_date'])
df_submission['dow'] = df_submission['visit_date'].dt.dayofweek
df_submission['year'] = df_submission['visit_date'].dt.year
df_submission['month'] = df_submission['visit_date'].dt.month
df_submission['visit_date'] = df_submission['visit_date'].dt.date

In [35]:
df_submission.head()

,id,visitors,visit_date,air_store_id,dow,year,month
0,air_00a91d42b08b08d9_2017-04-23,0,2017-04-23,air_00a91d42b08b08d9,6,2017,4
1,air_00a91d42b08b08d9_2017-04-24,0,2017-04-24,air_00a91d42b08b08d9,0,2017,4
2,air_00a91d42b08b08d9_2017-04-25,0,2017-04-25,air_00a91d42b08b08d9,1,2017,4
3,air_00a91d42b08b08d9_2017-04-26,0,2017-04-26,air_00a91d42b08b08d9,2,2017,4
4,air_00a91d42b08b08d9_2017-04-27,0,2017-04-27,air_00a91d42b08b08d9,3,2017,4


In [37]:
unique_stores = df_submission['air_store_id'].unique()

In [39]:
# what is this for?

stores = pd.concat([pd.DataFrame({'air_store_id': unique_stores, 'dow': [i]*len(unique_stores)}) for i in range(7)], 
                   axis=0, ignore_index=True).reset_index(drop=True)

In [41]:
stores.head(15)

,air_store_id,dow
0,air_00a91d42b08b08d9,0
1,air_0164b9927d20bcc3,0
2,air_0241aa3964b7f861,0
3,air_0328696196e46f18,0
4,air_034a3d5b40d5b1b1,0
5,air_036d4f1ee7285390,0
6,air_0382c794b73b51ad,0
7,air_03963426c9312048,0
8,air_04341b588bde96cd,0
9,air_049f6d5b402a31b2,0


In [44]:
stores[stores.air_store_id == 'air_00a91d42b08b08d9']

,air_store_id,dow
0,air_00a91d42b08b08d9,0
821,air_00a91d42b08b08d9,1
1642,air_00a91d42b08b08d9,2
2463,air_00a91d42b08b08d9,3
3284,air_00a91d42b08b08d9,4
4105,air_00a91d42b08b08d9,5
4926,air_00a91d42b08b08d9,6
